# AI PROJECT (MODEL # 1)
**MEMBERS:
Myer Ali Iqbal (17950)
Muhammad Affan (18019)
Hammad Shahid (18006)**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
store = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv")
train = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv")
test = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv",parse_dates=[3])

In [ ]:
train.shape


In [ ]:
train.head()

In [ ]:
test.shape

In [ ]:
test.head()


In [ ]:
store.head()


In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.describe()[['Sales','Customers']].loc['min'] #lower Bound of train set based on Sales and Customers

In [ ]:
train.describe()[['Sales','Customers']].loc['max'] #Upper Bound of train set based on Sales and Customers

In [ ]:
train.Store.nunique() #Total number of stores

In [ ]:
train['Store'].value_counts().head(50).plot.bar()

In [ ]:
train.DayOfWeek.value_counts()

In [ ]:
train.Open.value_counts()

In [ ]:
train.isna().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#date wise line plot for sales
train['Date'] = pd.to_datetime(train['Date'],format = '%Y-%m-%d')
store_id = train.Store.unique()[0]
print(store_id)
store_rows = train[train['Store'] == store_id]
print(store_rows.shape)
store_rows.resample('1D',on = 'Date')['Sales'].sum().plot.line(figsize = (18,8))

In [ ]:
store_rows[store_rows['Sales']==0] #missing values on days

In [ ]:
# checking the same for test data
test['Date'] = pd.to_datetime(test['Date'],format = '%Y-%m-%d')
store_test_rows = test[test['Store'] == store_id]
print(store_test_rows.shape)
store_test_rows['Date'].min(), store_test_rows['Date'].max()

In [ ]:
store_rows['Sales'].plot.hist(figsize = (14,8))

In [ ]:
store.isnull().sum() # Store data

In [ ]:
store.head()

In [ ]:
store[store['Store']==store_id].T # here store id was 1

In [ ]:
store[~store['Promo2SinceYear'].isna()].iloc[0] # checking the non null values in store data to make sure what we can fill in the missing values

**NOW WE'll TRY TO FILL THE MISSING VALUES ****

In [ ]:
#method 1
store['Promo2SinceWeek'].fillna(0,inplace = True)
store['Promo2SinceYear'].fillna(store['Promo2SinceYear'].mode()[0],inplace = True)
store['PromoInterval'].fillna(store['PromoInterval'].mode()[0],inplace = True)

In [ ]:
store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].mode()[0],inplace = True)
store['CompetitionDistance'].fillna(store['CompetitionDistance'].max(),inplace = True)
store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].mode()[0],inplace = True)

In [ ]:
store.isnull().sum()

In [ ]:
# merge the data in "train" and "store" sets
data_merged = train.merge(store,on = 'Store',how = 'left')
train.shape
data_merged.shape

In [ ]:
data_merged.isnull().sum().sum() # cross check if there are any missing values

In [ ]:
data_merged.dtypes
# 4 categorical columns
# 1 date columns 
# 13 numerical columns

In [ ]:
data_merged['day'] = data_merged['Date'].dt.day
data_merged['month'] = data_merged['Date'].dt.month
data_merged['year'] = data_merged['Date'].dt.year
#data_merged['weekday'] = data_merged['Date'].dt.strftime(%a)  This is already in data

In [ ]:
# stateHoliday, StoreType, Assortment, PromoInterval
data_merged['StateHoliday'].unique()

In [ ]:
data_merged['StateHoliday'] = data_merged['StateHoliday'].map({'a':1,'b':2,'c':3,'0':0,0:0})
data_merged['StateHoliday'] = data_merged['StateHoliday'].astype(int)

In [ ]:
pd.set_option('display.max_columns',None)
data_merged.head()

In [ ]:
data_merged['Assortment'] = data_merged['Assortment'].map({'a':1,'b':2,'c':3})
data_merged['Assortment'] = data_merged['Assortment'].astype(int)

In [ ]:
data_merged['StoreType'] = data_merged['StoreType'].map({'a':1,'b':2,'c':3,'d':4})
data_merged['StoreType'] = data_merged['StoreType'].astype(int)

In [ ]:
map_promo = {'Jan,Apr,Jul,Oct':1,'Feb,May,Aug,Nov':2,'Mar,Jun,Sept,Dec':3}
data_merged['PromoInterval'] = data_merged['PromoInterval'].map(map_promo)

WE WILL NOW SPLIT THE TRAIN SET TO CHECK THE ACCURACY OF OUR INITAIL MODEL. 
WE WILL THEN USE THE TEST DATA TO MAKE THE FINAL MODEL

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features = data_merged.columns.drop(['Sales','Date'])
X = data_merged[features]
y = np.log(data_merged['Sales']+1)
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 1) # ideal split is 80 20

DECISION TREE FOR THE MODEL

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
model_dt = DecisionTreeRegressor(max_depth = 20, random_state = 42).fit(X_train,y_train)

In [ ]:
y_pred = model_dt.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
#taken for stackOverflow https://stackoverflow.com/questions/53165807/how-to-calculate-rmspe-in-python-using-numpy
#root mean square percentage error (rmspe)
def rmspe(y, yhat):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

In [ ]:
y_inv = np.exp(y_test)-1
y_pred_inv = np.exp(y_pred)-1
np.sqrt(mean_squared_error(y_inv,y_pred_inv))
rmspe(y_inv,y_pred_inv)

In [ ]:
test.head()

# * **TEST DATA TESTING.....SORT OF**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
#plt.figure(figsize = (18,8))
#plt.bar(X,model_dt.feature_importances_)

In [ ]:
train_avg_cust = train.groupby(['Store'])[['Customers']].mean().reset_index().astype(int)

In [ ]:
test_1 = test.merge(train_avg_cust,on = 'Store',how = 'left')

In [ ]:
test.shape
test_1.shape

In [ ]:
test_1.head()

In [ ]:
test_merged = test_1.merge(store,on = 'Store',how = 'inner')
test_merged['Open'] = test_merged['Open'].fillna(1)
test_merged['Date'] = pd.to_datetime(test_merged['Date'],format = '%Y-%m-%d')
test_merged['day'] = test_merged['Date'].dt.day
test_merged['month'] = test_merged['Date'].dt.month
test_merged['year'] = test_merged['Date'].dt.year
test_merged['StateHoliday'] = test_merged['StateHoliday'].map({'0':0,'a':1})
test_merged['StateHoliday'] = test_merged['StateHoliday'].astype(int)
test_merged['Assortment'] = test_merged['Assortment'].map({'a':1,'b':2,'c':3})
test_merged['Assortment'] = test_merged['Assortment'].astype(int)
test_merged['StoreType'] = test_merged['StoreType'].map({'a':1,'b':2,'c':3,'d':4})
test_merged['StoreType'] = test_merged['StoreType'].astype(int)
map_promo = {'Jan,Apr,Jul,Oct':1,'Feb,May,Aug,Nov':2,'Mar,Jun,Sept,Dec':3}
test_merged['PromoInterval'] = test_merged['PromoInterval'].map(map_promo)

In [ ]:
test_merged.head()

In [ ]:
features = data_merged.columns.drop(['Sales','Customers','Date'])
X = data_merged[features]
y = np.log(data_merged['Sales']+1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
model_dt  = DecisionTreeRegressor(max_depth = 12, random_state = 1).fit(X_train,y_train)
y_pred = model_dt.predict(X_test)

In [ ]:
y_inv = np.exp(y_test)-1
y_pred_inv = np.exp(y_pred)-1
np.sqrt(mean_squared_error(y_inv,y_pred_inv))

In [ ]:
rmspe(y_inv,y_pred_inv)

In [ ]:
test_pred = model_dt.predict(test_merged[features])
test_pred_inv = np.exp(test_pred)-1


In [ ]:
dtrain = xgb.DMatrix(X_train[features],y_train)
dtest = xgb.DMatrix(X_test[features],y_test)

In [ ]:
params = {'max_depth':8,'eta':0.3,'objective':'reg:linear','subsample': 0.7,"colsample_bytree": 0.7,
         "silent": 1}
model_xg = xgb.train(params,dtrain, 300)

In [ ]:
y_pred = model_xg.predict(dtest)

In [ ]:
y_inv = np.exp(y_test) -1
y_pred_inv = np.exp(y_pred) - 1

In [ ]:
rmspe(y_inv, y_pred_inv)

In [ ]:
mean_squared_error(y_inv, y_pred_inv)

In [ ]:
mean_absolute_error(y_inv, y_pred_inv)

In [ ]:
r2_score(y_inv, y_pred_inv)

In [ ]:
testdmatrix = xgb.DMatrix(test_merged[X_train.columns])

In [ ]:
y_pred_log = model_xg.predict(testdmatrix)

In [ ]:
y_pred_exp = np.exp(y_pred_log) - 1
y_pred_exp 

In [ ]:
test['Sales'] = y_pred_exp

In [ ]:
sales_correct= []
for i in range(len(y_pred_exp)):
    if i in test[test['Open'] == 0].index:
        sales_correct.append(0)
    else:
        sales_correct.append(test['Sales'][i])


In [ ]:
test['Sales'] = sales_correct

In [ ]:
test.head()